# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [ ]:
import math

import numpy as np
import pandas as pd

Загрузим данные и проведем предобраотку данных как на семинаре.

In [ ]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

In [ ]:
articles_df = pd.read_csv("articles/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [ ]:
interactions_df = pd.read_csv("articles/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [ ]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [ ]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(
    lambda x: event_type_strength[x]
)

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [ ]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [ ]:
interactions_from_selected_users_df = interactions_df.loc[
    np.in1d(interactions_df.personId, users_with_enough_interactions_df)
]

In [ ]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [ ]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [ ]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [ ]:
interactions = (
    interactions_train_df.groupby("personId")["contentId"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = interactions_test_df.groupby("personId")["contentId"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    ""
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [ ]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.data import Dataset

## Задание 1 (1.5 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [ ]:
# Ваш код здесь
def lightfm_dataset(df, user, item):
    data = Dataset()
    data.fit(df[user], df[item])
    return data

In [ ]:
data = lightfm_dataset(interactions_full_df, "personId", "contentId")

data_train = data.build_interactions(interactions_train_df[["personId", "contentId", "eventStrength"]].apply(tuple, axis=1))[1]
data_test = data.build_interactions(interactions_test_df[["personId", "contentId", "eventStrength"]].apply(tuple, axis=1))[1]

## Задание 2 (0.5 балла)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [ ]:
# Ваш код здесь
lightFM = LightFM(loss = 'warp', random_state=42)

lightFM.fit(data_train, epochs=50)

precision_at_k(lightFM, data_test, data_train, 10).mean()

0.0071283095

## Задание 3 (2 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [ ]:
# Ваш код здесь
from sklearn.feature_extraction.text import TfidfVectorizer

df_text = pd.merge(interactions_full_df['contentId'].drop_duplicates(), 
                   articles_df[['contentId', 'text']], 
                   how='left', on='contentId').fillna("")

tf_idf = TfidfVectorizer()

feat = tf_idf.fit_transform(df_text['text'])

feat

<2984x71875 sparse matrix of type '<class 'numpy.float64'>'
	with 1041129 stored elements in Compressed Sparse Row format>

In [ ]:
lightFM_feat = LightFM(loss = 'warp', random_state=42)

lightFM_feat.fit(data_train, epochs=50, item_features=feat)
                                   
precision_at_k(lightFM_feat, data_test, data_train, 10, item_features=feat).mean()

0.007942974

## Задание 4 (1.5 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [ ]:
!pip install pymorphy2
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

from multiprocessing import Pool
from tqdm import tqdm

import nltk
nltk.download('stopwords')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
articles_df['lang'].value_counts()

en    2211
pt     829
la       3
es       2
ja       2
Name: lang, dtype: int64

In [ ]:
m = MorphAnalyzer()
regex = re.compile("\w+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = [*stopwords.words('english'), *stopwords.words('portuguese')]
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [ ]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df_text['text']), total=len(df_text)))
    
df_text['lemmas'] = lemmas
df_text.sample(5)

100%|██████████| 2984/2984 [01:02<00:00, 47.86it/s]


,contentId,text,lemmas
1421,2399228553079505025,"Three years ago, our venture capital firm bega...",three years venture capital firm began studyin...
2059,-5444306124868253918,Os CEOs não estão suscetíveis em reduzir os ga...,ceos suscetíveis reduzir gastos tecnologia fac...
2810,4532698165746826599,And how DevOps can help Your company decides t...,devops help company decides bring digital expe...
2528,-5786811453218931415,Muitos jovens têm ideias inovadoras e uma vont...,muitos jovens ideias inovadoras vontade enorme...
132,-4531029390350541169,This isn't exactly about a feature. Features a...,exactly feature features verbs mobile product ...


In [ ]:
feat2 = tf_idf.fit_transform(df_text['lemmas'])

feat2

<2984x67392 sparse matrix of type '<class 'numpy.float64'>'
	with 805885 stored elements in Compressed Sparse Row format>

In [ ]:
lightFM_feat_norm = LightFM(loss = 'warp', random_state=42)

lightFM_feat_norm.fit(data_train, epochs=50, item_features=feat2)
                                   
precision_at_k(lightFM_feat_norm, data_test, data_train, 10, item_features=feat2).mean()

0.008452139

Улучшилось ли качество предсказания?

**Ответ:** *качество предсказания в принципе не очень высокое, но получше, чем без предобработки текста*

## Задание 5 (1.5 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [ ]:
# Ваш код здесь
best = 0
best_num = 0
best_params = dict()

for i in tqdm(range(20)):
    np.random.seed(i)
    # Значения параметров для перебора отсюда: https://stackoverflow.com/questions/49896816/how-do-i-optimize-the-hyperparameters-of-lightfm
    # в параметры распределений передавались базовые параметры LightFM
    params = {"no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "max_sampled": np.random.randint(5, 15),
            }

    lightFM_opt = LightFM(**params, random_state=42)

    lightFM_opt.fit(data_train, epochs=50)
    precision_k = precision_at_k(lightFM_opt, data_test, data_train, 10).mean()
                                   
    params['precision_k'] = precision_k
    best_params[i] = params

    if precision_k > best:
        best = precision_k
        best_num = i


100%|██████████| 20/20 [01:35<00:00,  4.77s/it]


In [ ]:
def without_keys(d, keys):
    return {k: v for k, v in d.items() if k not in keys}

par = without_keys(best_params[best_num], 'precision_k')


lightFM_opt = LightFM(**par, random_state=42)

lightFM_opt.fit(data_train, epochs=50, item_features=feat2)

In [ ]:
precision_at_k(lightFM_opt, data_test, data_train, 10, item_features=feat2).mean()

0.009063136

## Задание 6 (1 балл)

Реализуйте функции для вычисления следующих метрик:
* precision@k
* recall@k
* NDCG@k



In [ ]:
# для отдельного пользователя
def single_precision(pred_values, true_values, k):
    top_k = pred_values[:k]
    TP = np.in1d(top_k, true_values).sum()
    return TP / k
    # return TP / min(len(true_values) + 0.0001, k)

def single_recall(pred_values, true_values, k):
    top_k = pred_values[:k]
    TP = np.in1d(top_k, true_values).sum()
    return TP / len(true_values)

def single_ndcg(pred_values, true_values, k):
    top_k = pred_values[:k]
    sorted_true_values = np.argsort(true_values)[::-1]
    DCG = 0
    IDCG = 0

    for i_num, i in enumerate(top_k):
        # скор предсказанного айтема с юзером в тесте
        score = true_values[i]
        # позиция данного айтема в отсортированном тесте
        pos = np.where(sorted_true_values == i)[0]
        g = 2**score - 1
        d = 1 / np.log2(pos+1+1)[0] # +1 чтобы не делить на ноль (с 1 начинается список предсказаний)
        DCG += g * d

        # скор реальных самых релевантных айтемов
        score_idcg = true_values[sorted_true_values[i_num]]
        # порядковый номер реальных самых релевантных айтемов
        pos_idcg = i_num
        g_idcg = 2**score_idcg - 1
        d_idcg = 1 / np.log2(pos_idcg+1+1)
        IDCG += g_idcg * d_idcg

    return DCG/IDCG

# для всех пользователей
def my_precision_k(df, recommendations_col, true_values_col, k):
    return df.apply(lambda row: single_precision(row[recommendations_col], row[true_values_col], k), axis=1).mean()
        
def my_recall_k(df, recommendations_col, true_values_col, k):
    return df.apply(lambda row: single_recall(row[recommendations_col], row[true_values_col], k), axis=1).mean()

def my_nDCG_k(df, recommendations_col, true_values, k):
    return df.apply(lambda row: single_ndcg(row[recommendations_col], true_values.toarray()[row.name, :], k), axis=1).mean()

## Задание 7 (1 балл)

Вычислите значения реализованных метрик для $k=10$ для лучшей полученной модели в предыдущих шагах.

Найдите уже реализованные варианты этих метрик в библиотеках lightfm и sklearn. Сравните полученные у вас значения метрик с результатами встроенных в библиотеки метрик.

In [ ]:
# Для предсказания по лучшей модели без уже просмотренных

# на выходe: список айтемов отсортированных по recommendation score и список отсортированных recommendation score
def predict_without_prev_values(model, prev_matrix, pred_matrix, user_matrix_id, item_features):
    # пользователь уже читал
    prev_values = prev_matrix.toarray()[user_matrix_id, :].nonzero()[0]

    # рекомендации пользователю
    pr = model.predict(user_matrix_id, np.arange(pred_matrix.shape[1]), item_features=item_features)
    recommend = np.argsort(pr)[::-1] # сортируем рекомендации

    # убираем из рекомендаций те, что уже читал
    recommend = recommend[~np.in1d(recommend, prev_values)]
    # recommend[np.in1d(recommend, prev_values)] = -

    return recommend


In [ ]:
# получаем рекомендации для каждого пользователя
recommendations = []
for id in range(data_test.shape[0]):
    single_rec = predict_without_prev_values(lightFM_opt, data_train, data_test, id, feat2)
    recommendations.append(single_rec)

def insertion(array):
    if len(array) == 0:
        return None
    else:
        return array

# датафрейм из предсказаний и реальных значений
df_res = pd.DataFrame({'recommendations': recommendations, 
                       'true_values':     [np.nonzero(t)[0] for t in data_test.toarray()]})

# уберем тех пользователей, у которых список реальных значений пуст
df_res['true_values'] = df_res['true_values'].apply(lambda row: insertion(row))
df_res = df_res[~df_res['true_values'].isna()]
df_res.head()

,recommendations,true_values
0,"[959, 655, 966, 223, 824, 1244, 879, 884, 210,...","[1, 3, 4, 5]"
1,"[1788, 2718, 2354, 2112, 2052, 968, 766, 662, ...","[16, 51, 58, 60, 61, 75, 76, 97, 102, 121, 123..."
2,"[652, 1284, 600, 1566, 2507, 546, 1580, 580, 1...","[58, 131, 401, 559, 667, 669, 676, 683, 693, 7..."
3,"[824, 655, 959, 989, 223, 343, 300, 613, 968, ...","[795, 796, 797, 798, 799, 800]"
4,"[122, 419, 1604, 62, 666, 621, 322, 1733, 282,...","[131, 364, 559, 750, 804, 805, 807, 809, 812, ..."


In [ ]:
print(f"my_precision@k: {my_precision_k(df_res, 'recommendations', 'true_values', k=10):.6f}")
print(f"my_recall@k:    {my_recall_k(df_res, 'recommendations', 'true_values', k=10):.6f}")
print(f"my_nDCG@k:      {my_nDCG_k(df_res, 'recommendations', data_test, k=10):.6f}")

my_precision@k: 0.009063
my_recall@k:    0.011647
my_nDCG@k:      0.020723


In [ ]:
from lightfm.evaluation import recall_at_k
from sklearn.metrics import ndcg_score

print(f"lightFM precision@k: {precision_at_k(lightFM_opt, data_test, data_train, k=10, item_features=feat2).mean():.6f}")
print(f"lightFM recall@k:    {recall_at_k(lightFM_opt, data_test, data_train, k=10, item_features=feat2).mean():.6f}")

# поправлю функцию, чтобы получать скоры в предикте, а не сразу нужный порядок
def predict_scores(model, prev_matrix, pred_matrix, user_matrix_id):
    # пользователь уже читал
    prev_values = prev_matrix.toarray()[user_matrix_id, :]
    prev_values = np.where(prev_values != 0, False, True)

    # скоры рекомендаций пользователю
    pr = model.predict(user_matrix_id, np.arange(pred_matrix.shape[1]))

    return pr[prev_values]

sk_ndcg = df_res.apply(lambda row: ndcg_score(
                        np.asarray([data_test.toarray()[row.name, :][np.where([data_train.toarray()[row.name, :] == 0])[1]]]), 
                        np.asarray([predict_scores(lightFM_opt, data_train, data_test, row.name)]), k=10), axis=1).mean()

print(f"sklearn nDCG@k:      {sk_ndcg:.6f}")


lightFM precision@k: 0.009063
lightFM recall@k:    0.011647
sklearn nDCG@k:      0.001318


*Результаты самописных метрик сходятся со встроенными. (но в precision@k - используется в знаменателе значение k, вне зависимости от кол-ва реальных значений. делал замену на min(len(true_values) + 0.0001, k) [закомментировано в коде] и сразу были расхождения, хотя кажется что так правильнее)*

*Расхождение есть в nDCG, но объясняется используемой внутри sklearn метрики формулой, отличающуюся от той, что давали на лекции. Решил делать по той, которую понял + наверно по-другому работает со значениями, по которым нет тестовых наблюдений, я же их оставлял*

## Задание 8 (1 балл)

Реализуйте алгоритм ALS и примените его для решения задачи ноутбука.

**ALS**

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}$$

Суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя~$u$ и товара $i$, соответственно, матрицы $P, Q$ получаются путем записывания по столбцам векторов $p_u, q_i$ соответственно.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$

Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$

аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья c постановкой задачи для ALS на explicit feedback:**

* Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.

**Оригинальная статья с ALS для implicit данных, которая стала более известной:**

* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee.


In [ ]:
!pip install implicit
import implicit 
import scipy.sparse as sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

data_to_fit = (data_train * 40).astype('double') 
model.fit(data_to_fit)

/usr/local/lib/python3.9/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0006892681121826172 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
recs = []
for i in np.arange(data_test.shape[0]):
    recs.append(model.recommend(i, data_test.tocsr()[i], filter_already_liked_items=False)[0])


In [ ]:
df_als = pd.DataFrame({'recommendations': recs, 
                       'true_values':     [np.nonzero(t)[0] for t in data_test.toarray()]})

df_als['true_values'] = df_als['true_values'].apply(lambda row: insertion(row))
df_als = df_als[~df_als['true_values'].isna()]

In [ ]:
print(f"my_precision@k: {my_precision_k(df_als, 'recommendations', 'true_values', k=10):.6f}")
print(f"my_recall@k:    {my_recall_k(df_als, 'recommendations', 'true_values', k=10):.6f}")
print(f"my_nDCG@k:      {my_nDCG_k(df_als, 'recommendations', data_test, k=10):.6f}")

my_precision@k: 0.006314
my_recall@k:    0.006708
my_nDCG@k:      0.011967
